# FA691 Homework 3
# Due: Wednesday, February 15 @ 11:59PM

Name: 

Date: 

In [1]:
import numpy as np
import yfinance
import matplotlib.pyplot as plt

# Set seed of random number generator
CWID = -1 #Place here your Campus wide ID number, this will personalize
#your results, but still maintain the reproduceable nature of using seeds.
#If you ever need to reset the seed in this assignment, use this as your seed
#Papers that use -1 as this CWID variable will earn 0's so make sure you change
#this value before you submit your work.
personal = CWID % 10000
np.random.seed(personal)

## Question 1 (10pt)

### Question 1.1
Use the `yfinance` package (or other method of your choice) to obtain the daily adjusted close prices for `SPY` and `IEF`.  You should have at least 5 years of data for both assets. Do **not** include any data after January 1, 2022.  You should inspect the dates for your data to make sure you are including everything appropriately.  Create a binary variable whether the `SPY` returns are above the `IEF` returns on a each day. Create a data frame (or array) of the daily log returns both both stocks along with the lagged returns (at least 2 lags) and your binary class variable.  Use the `print` command to display your data.

In [2]:
# Enter your code here

### Question 1.2
Split your data into training and testing sets (80% training and 20% test). This split should be done so that the causal relationship is kept consistent (i.e., split data at a specific time).

Run a logistic regression of the binary variable (of `SPY` returns greater than `IEF` returns) as a function of the lagged returns (at least 2 lags) for both stocks.
This should be of the form (assuming 2 lags) of $p_{t} = [1 + \exp(-[\beta_0 + \beta_{SPY,1} r_{SPY,t-1} + \beta_{SPY,2} r_{SPY,t-2} + \beta_{IEF,1} r_{IEF,t-1} + \beta_{IEF,2} r_{IEF,t-2}])]^{-1}$.
Evaluate the performance of this model by printing the confusion matrix and accuracy on the test data.

In [3]:
# Enter your code here

<font color='red'>Solution:</font>

## Question 2 (30spt)

### Question 2.1
Using the same data, train/test split ratio, and consider the same classification problem as in Question 1.2.
Create a feed-forward neural network with a single hidden layer (10 hidden nodes) densely connected to the inputs.
You may choose any activation functions you wish.

In [4]:
# Enter your code here

### Question 2.2
Train this neural network on the training data.  
Evaluate the performance of this model by printing the confusion matrix and accuracy on the test data.

In [5]:
# Enter your code here

<font color='red'>Solution:</font> 

### Question 2.3
Using the same train/test split and consider the same classification problem as in Question 1.2.
Train and test another feed-forward neural network of your own design.

In [6]:
# Enter your code here

<font color='red'>Solution:</font>

## Question 3 (30pt)

### Question 3.1
Using the same data, train/test split ratio, and consider the same classification problem as in Question 1.2.
Create a time dilation neural network with a single convolutional layer (filter size of 10, kernel size of 2, dilation size of 1) densely connected to the inputs.
You may choose any activation functions you wish.

*Hint:* The CNN can reference earlier lags on its own without feeding explicit memory inputs as was needed for the Question 2.

In [7]:
# Enter your code here

### Question 3.2
Train this neural network on the training data.
Evaluate the performance of this model by printing the confusion matrix and accuracy on the test data.

In [8]:
# Enter your code here

<font color='red'>Solution:</font> 

### Question 3.3
Using the same train/test split and consider the same classification problem as in Question 1.2. Train and test another convolutional neural network of your own design.

In [9]:
# Enter your code here

<font color='red'>Solution:</font> 

## Question 4 (30pt)

## Question 4.1
Consider the same classification problem as in Question 1.2.
Of the methods considered in this assignment, which would you recommend in practice?
Explain briefly (1 paragraph) why you choose this fit. 

<font color='red'>Solution:</font> 

## Question 4.2
Recreate your data set using data from January 1, 2022 through December 31, 2022.
Using the method your would implement in practice, invest in the asset (``SPY`` or ``IEF``) depending on your predictions.
Print the returns your portfolio would obtain from following this strategy. Comment on how this portfolio compares with the ``SPY`` and ``IEF`` returns and risks.

In [10]:
# Enter your code here

<font color='red'>Solution:</font> 